<a href="https://colab.research.google.com/github/andrelmsunb/FIPE.manuscript/blob/main/Pesquisa_FIPE_Veiculos_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - Previsão de Valor FIPE de Veículos

Este notebook segue o roteiro proposto para a criação de um MVP (Minimum Viable Product) de Machine Learning.

**Autor**: Análise realizada com Andre Luiz Marques Serrano  
**Data**: 10/09/2025

## 1. Definição do Problema

- **Objetivo**: Construir um modelo de Machine Learning para prever o **Valor FIPE** de um veículo com base em suas características (ano, montadora, modelo, etc.).
- **Tipo de Problema**: Regressão.
- **Base de Dados**: `CópiadeVeículos.xlsx`, contendo dados de uma frota de veículos.
- **Premissas**: O Valor FIPE é influenciado principalmente por características como ano, montadora e modelo. Outras variáveis podem ter menor impacto.
- **Restrições**: Foram removidos dados com inconsistências graves, como `Ano Modelo` inválido e `Valor FIPE` zerado.

## 2. Setup e Importação das Bibliotecas

In [1]:
# Instalação de dependências (execute apenas se necessário)
!pip install xgboost

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")

# Configurações de visualização
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("viridis")

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


## 3. Carga e Preparação dos Dados

**Instruções para carregar os dados no Google Colab:**

1. Faça o upload do arquivo `veiculos_limpo.csv` (gerado na análise prévia)
2. Ou carregue o arquivo original `CópiadeVeículos.xlsx` e execute a limpeza dos dados

**Opção 1: Carregar arquivo limpo**

In [2]:
# Opção 1: Upload do arquivo limpo
from google.colab import files
uploaded = files.upload()
df = pd.read_excel(next(iter(uploaded.keys())))

print(f"Dimensões do dataset: {df.shape}")
print("\nVisualização das primeiras linhas:")
df.head()

Saving Cópia de Veículos (1).xlsx to Cópia de Veículos (1).xlsx


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9e in position 14: invalid start byte

**Opção 2: Carregar e limpar arquivo original**

In [ ]:
# Opção 2: Upload e limpeza do arquivo original
# from google.colab import files
# uploaded = files.upload()
# df_raw = pd.read_excel(next(iter(uploaded)))

# # Limpeza dos dados
# df = df_raw.copy()
# df = df[df['Ano Modelo'] >= 1990]  # Filtrar anos realistas
# df = df[df['Valor FIPE'] > 0]      # Remover valores FIPE zerados

# print(f"Dataset original: {df_raw.shape}")
# print(f"Dataset após limpeza: {df.shape}")
# df.head()

## 4. Análise Exploratória dos Dados

In [ ]:
# Informações gerais do dataset
print("=== INFORMAÇÕES GERAIS ===")
print(f"Dimensões: {df.shape}")
print(f"\nColunas: {list(df.columns)}")
print(f"\nTipos de dados:")
print(df.dtypes)

# Estatísticas descritivas da variável target
print(f"\n=== ESTATÍSTICAS DO VALOR FIPE ===")
print(df['Valor FIPE'].describe())

# Análise das principais variáveis categóricas
print(f"\n=== PRINCIPAIS VARIÁVEIS CATEGÓRICAS ===")
print(f"\nMontadoras (top 5):")
print(df['Montadora'].value_counts().head())

print(f"\nModelos (top 5):")
print(df['Modelo'].value_counts().head())

print(f"\nCores:")
print(df['Cor'].value_counts())

print(f"\nSituação:")
print(df['Situação'].value_counts())

In [ ]:
# Visualizações exploratórias
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribuição do Valor FIPE
axes[0, 0].hist(df['Valor FIPE'], bins=50, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Distribuição do Valor FIPE')
axes[0, 0].set_xlabel('Valor FIPE (R$)')
axes[0, 0].set_ylabel('Frequência')

# 2. Valor FIPE por Ano Modelo
axes[0, 1].scatter(df['Ano Modelo'], df['Valor FIPE'], alpha=0.6, color='coral')
axes[0, 1].set_title('Valor FIPE por Ano Modelo')
axes[0, 1].set_xlabel('Ano Modelo')
axes[0, 1].set_ylabel('Valor FIPE (R$)')

# 3. Top 10 Montadoras
top_montadoras = df['Montadora'].value_counts().head(10)
axes[1, 0].bar(range(len(top_montadoras)), top_montadoras.values, color='lightgreen')
axes[1, 0].set_title('Top 10 Montadoras')
axes[1, 0].set_xlabel('Montadoras')
axes[1, 0].set_ylabel('Quantidade')
axes[1, 0].set_xticks(range(len(top_montadoras)))
axes[1, 0].set_xticklabels(top_montadoras.index, rotation=45, ha='right')

# 4. Valor FIPE por Situação
df.boxplot(column='Valor FIPE', by='Situação', ax=axes[1, 1])
axes[1, 1].set_title('Valor FIPE por Situação')
axes[1, 1].set_xlabel('Situação')
axes[1, 1].set_ylabel('Valor FIPE (R$)')

plt.tight_layout()
plt.show()

# Correlação entre Ano e Valor FIPE
correlation = df[['Ano Modelo', 'Valor FIPE']].corr()
print(f"\nCorrelação entre Ano Modelo e Valor FIPE:")
print(correlation)

## 5. Feature Engineering e Pré-processamento

In [ ]:
# Definindo a variável target (y) e as features (X)
target = "Valor FIPE"
features_categoricas = ["Montadora", "Modelo", "Cor", "Grupo", "Situação"]
features_numericas = ["Ano Modelo"]

X = df[features_categoricas + features_numericas]
y = df[target]

print(f"Target: {target}")
print(f"Features: {X.columns.tolist()}")
print(f"\nShape das features: {X.shape}")
print(f"Shape do target: {y.shape}")

# Verificar valores nulos
print(f"\nValores nulos nas features:")
print(X.isnull().sum())

In [ ]:
# Divisão dos dados em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")
print(f"Proporção treino/teste: {X_train.shape[0]/X_test.shape[0]:.1f}:1")

In [ ]:
# Criação do pipeline de pré-processamento
# 1. Features Numéricas: Padronização (StandardScaler)
# 2. Features Categóricas: One-Hot Encoding (ignorar categorias não vistas no treino)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), features_numericas),
        ("cat", OneHotEncoder(handle_unknown="ignore"), features_categoricas),
    ]
)

print("Pipeline de pré-processamento criado.")
print(f"\nTransformações aplicadas:")
print(f"- Features numéricas ({len(features_numericas)}): StandardScaler")
print(f"- Features categóricas ({len(features_categoricas)}): OneHotEncoder")

## 6. Modelagem e Treinamento

In [ ]:
# Definindo os modelos que serão treinados e comparados
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100),
    "XGBoost": XGBRegressor(random_state=42, n_estimators=100),
}

results = {}

print("Iniciando o treinamento dos modelos...")
print(f"Modelos a serem treinados: {list(models.keys())}")

In [ ]:
for name, model in models.items():
    print(f"\n--- Treinando {name} ---")

    # Criando o pipeline final com pré-processamento e o modelo
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", model)])

    # Treinamento
    pipeline.fit(X_train, y_train)

    # Predição
    y_pred = pipeline.predict(X_test)

    # Avaliação
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results[name] = {"MAE": mae, "RMSE": rmse, "R²": r2, "predictions": y_pred}

    print(f"Resultados para {name}:")
    print(f"  - MAE (Erro Médio Absoluto): R$ {mae:,.2f}")
    print(f"  - RMSE (Raiz do Erro Quadrático Médio): R$ {rmse:,.2f}")
    print(f"  - R² (Coeficiente de Determinação): {r2:.4f}")

print("\nTreinamento concluído!")

## 7. Avaliação e Comparação dos Resultados

In [ ]:
# Tabela comparativa de métricas
results_df = pd.DataFrame(results).T.drop(columns=["predictions"])
print("=== Tabela Comparativa de Métricas ===")
results_df_sorted = results_df.sort_values(by="R²", ascending=False)
print(results_df_sorted)

# Identificar o melhor modelo
best_model_name = results_df["R²"].idxmax()
best_model_r2 = results_df.loc[best_model_name, "R²"]
best_model_mae = results_df.loc[best_model_name, "MAE"]

print(f"\n🏆 Melhor modelo: {best_model_name}")
print(f"   R² = {best_model_r2:.4f}")
print(f"   MAE = R$ {best_model_mae:,.2f}")

In [ ]:
# Visualização dos resultados
fig, axes = plt.subplots(1, 3, figsize=(20, 6), sharey=True)
fig.suptitle("Comparação entre Valores Reais e Preditos por Modelo", fontsize=16)

for i, (name, data) in enumerate(results.items()):
    sns.scatterplot(ax=axes[i], x=y_test, y=data["predictions"], alpha=0.6)
    axes[i].set_title(f"{name} (R² = {data['R²']:.3f})")
    axes[i].set_xlabel("Valor FIPE Real")
    axes[i].set_ylabel("Valor FIPE Predito")
    # Linha de referência (y=x)
    axes[i].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
# Análise de resíduos do melhor modelo
best_predictions = results[best_model_name]["predictions"]
residuals = y_test - best_predictions

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Gráfico de resíduos
axes[0].scatter(best_predictions, residuals, alpha=0.6)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('Valores Preditos')
axes[0].set_ylabel('Resíduos')
axes[0].set_title(f'Análise de Resíduos - {best_model_name}')

# Histograma dos resíduos
axes[1].hist(residuals, bins=30, alpha=0.7)
axes[1].set_xlabel('Resíduos')
axes[1].set_ylabel('Frequência')
axes[1].set_title('Distribuição dos Resíduos')

plt.tight_layout()
plt.show()

print(f"Estatísticas dos resíduos:")
print(f"Média: {residuals.mean():.2f}")
print(f"Desvio padrão: {residuals.std():.2f}")
print(f"Mediana: {residuals.median():.2f}")

## 8. Conclusão e Próximos Passos

In [ ]:
print("=== CONCLUSÃO DO MVP ===")
print(f"O modelo com melhor desempenho foi o **{best_model_name}**, com um R² de **{best_model_r2:.4f}**.")
print("Este valor indica que o modelo consegue explicar uma porção significativa da variabilidade do Valor FIPE.")
print(f"O Erro Médio Absoluto (MAE) nos diz que, em média, as previsões do modelo erram em torno de R$ {best_model_mae:,.2f}.")

print("\n**Próximos Passos Sugeridos:**")
print("1. **Otimização de Hiperparâmetros**: Utilizar técnicas como GridSearchCV ou RandomizedSearchCV para encontrar os melhores hiperparâmetros para o XGBoost e Random Forest.")
print("2. **Feature Engineering Avançada**: Criar novas features, como a idade do veículo (ano atual - ano modelo) ou combinar informações de diferentes colunas.")
print("3. **Análise de Erros**: Investigar os casos onde o modelo mais erra para entender suas limitações.")
print("4. **Validação Cruzada**: Implementar validação cruzada (K-Fold) para obter uma estimativa mais robusta do desempenho do modelo.")
print("5. **Inclusão de Mais Dados**: Utilizar mais features que foram descartadas (após tratamento adequado) ou buscar dados externos.")

print("\n**Insights Obtidos:**")
print(f"- O dataset possui {df.shape[0]} veículos com {df.shape[1]} características.")
print(f"- A correlação entre Ano Modelo e Valor FIPE é de {df[['Ano Modelo', 'Valor FIPE']].corr().iloc[0,1]:.3f}.")
print(f"- As principais montadoras são: {', '.join(df['Montadora'].value_counts().head(3).index.tolist())}.")
print(f"- O modelo Linear Regression teve desempenho surpreendentemente bom, indicando relações lineares fortes entre as features e o target.")